# 0) Dataload

In [1]:
# === 1. 설정 및 라이브러리 임포트 ===
import pandas as pd
import numpy as np
import random
import os
import itertools
from ast import literal_eval
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# -----------------------------------------
# [중요] 재현성을 위한 시드(Seed) 고정
# -----------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# 파일 경로 설정 (사용자 환경에 맞게 수정됨)
# DATA_PATH = "/Users/kangmingyun/Desktop/프로젝트/한식레시피/dataset/final/1125final(ing).csv"
DATA_PATH = "C:/Users/User/OneDrive/바탕 화면/한식레시피/data_final/1126final.csv"

print("✅ 라이브러리 임포트 및 시드 고정 완료")

✅ 라이브러리 임포트 및 시드 고정 완료


In [2]:
# === 2. 데이터 로드 ===
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ 파일이 없습니다: {DATA_PATH}")

print(f"📂 데이터 로딩 중: {DATA_PATH}")
df = pd.read_csv(DATA_PATH)

📂 데이터 로딩 중: C:/Users/User/OneDrive/바탕 화면/한식레시피/data_final/1126final.csv


In [3]:
# 불필요한 인덱스 컬럼 제거
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns='Unnamed: 0')

# [핵심] 문자열로 저장된 리스트를 실제 리스트 객체로 변환
# 예: "['마늘', '파']" -> ['마늘', '파']
print("🔄 재료 토큰 리스트 변환 중...")
df['재료토큰'] = df['재료토큰'].apply(literal_eval)

print(f"✅ 데이터 로드 완료! 총 레시피 수: {len(df):,}개")
df

🔄 재료 토큰 리스트 변환 중...
✅ 데이터 로드 완료! 총 레시피 수: 130,809개


,레시피일련번호,요리명,재료통합,요리종류별명_세분화,재료토큰,요리방법별명
0,128671.0,두부새우전,녹말가루 브로콜리 새우 계란 고추 두부 당근,밑반찬,"[녹말가루, 브로콜리, 새우, 계란, 고추, 두부, 당근]",부침
1,128932.0,알밥,계란 노른자 참기름 김가루 후리가케 날치 참깨 당근 단무지 무 밥 신김치,밥,"[계란, 노른자, 참기름, 김가루, 후리가케, 날치, 참깨, 당근, 단무지, 무, ...",굽기
2,131871.0,현미호두죽,물 소금 호두 현미 찹쌀,죽,"[물, 소금, 호두, 현미, 찹쌀]",끓이기
3,139247.0,북어갈비,간장 마늘 파 후춧가루 참기름 설탕 찹쌀가루 물 북어포 깨소금,메인반찬,"[간장, 마늘, 파, 후춧가루, 참기름, 설탕, 찹쌀가루, 물, 북어포, 깨소금]",굽기
4,149207.0,토마토스파게티,올리브유 양파 마늘 토마토 파스타면 피망,면,"[올리브유, 양파, 마늘, 토마토, 파스타면, 피망]",볶음
...,...,...,...,...,...,...
130804,7014698.0,떡볶이,양파 파 물 배추 고추 동전육수 참치액 김 허니부쉬 깨 깻잎,메인반찬,"[양파, 파, 물, 배추, 고추, 동전육수, 참치액, 김, 허니부쉬, 깨, 깻잎]",끓이기
130805,7014700.0,짜장떡볶이,상추 고추 밥 마요네즈 맛살 오이 단무지 후추 참기름 오징어,떡,"[상추, 고추, 밥, 마요네즈, 맛살, 오이, 단무지, 후추, 참기름, 오징어]",볶음
130806,7014702.0,누룽지야채계란죽,간장 설탕 올리고당 참기름 고추 소금 김 참치액젓 매실청 깨,죽,"[간장, 설탕, 올리고당, 참기름, 고추, 소금, 김, 참치액젓, 매실청, 깨]",끓이기
130807,7014705.0,오징어젓무침,밥 식용유 홍합 돼지고기 후추 스파게티 면 고추 토마토소스 치즈,젓갈,"[밥, 식용유, 홍합, 돼지고기, 후추, 스파게티, 면, 고추, 토마토소스, 치즈]",무침


In [4]:
df['요리종류별명_세분화'].unique() 

array(['밑반찬', '밥', '죽', '메인반찬', '면', '떡', '만두', '찌개', '소스', '국/탕', '양념',
       '퓨전', '잼', '김치', '젓갈', '장류'], dtype=object)

In [5]:
# 카테고리 통합 매핑 함수
def simplify_category(cat):
    if cat in ['국/탕', '찌개']:
        return '국물요리'
    elif cat in ['밑반찬', '메인반찬', '퓨전']:
        return '일품요리' # 반찬류 통합
    elif cat in ['밥', '죽', '면', '떡', '만두']:
        return '곡물분식'
    elif cat in ['김치', '젓갈', '장류']:
        return '저장음식'
    elif cat in ['양념', '소스', '잼']:
        return '소스류'
    else:
        return '기타'

# 적용
print("🔄 카테고리 단순화 중...")
df['요리종류별명_세분화'] = df['요리종류별명_세분화'].apply(simplify_category)

print("✅ 카테고리 재분류 완료:")
print(df['요리종류별명_세분화'].unique())

🔄 카테고리 단순화 중...
✅ 카테고리 재분류 완료:
['일품요리' '곡물분식' '국물요리' '소스류' '저장음식']


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130809 entries, 0 to 130808
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   레시피일련번호     130809 non-null  float64
 1   요리명         130809 non-null  object 
 2   재료통합        130809 non-null  object 
 3   요리종류별명_세분화  130809 non-null  object 
 4   재료토큰        130809 non-null  object 
 5   요리방법별명      130809 non-null  object 
dtypes: float64(1), object(5)
memory usage: 6.0+ MB


In [7]:
df['요리종류별명_세분화'].unique()

array(['일품요리', '곡물분식', '국물요리', '소스류', '저장음식'], dtype=object)

# 1) Word2vec & Doc2vec 학습

In [8]:
# === 3. 모델 학습 (Word2Vec + Doc2Vec) ===

# 3-1. Word2Vec 학습
print("🚀 Word2Vec 학습 시작... (Window=30, Min=3, Sample=1e-4)")
w2v_model = Word2Vec(
    sentences=df['재료토큰'],
    vector_size=100,
    window=30,        # 레시피 내 모든 재료 관계 학습
    min_count=3,      # 3번 이상 등장하면 학습 (다양성 확보)
    sg=1,             # Skip-gram
    # sample=1e-4,      # 빈출 단어 패널티 (핵심!)
    # ns_exponent=0.75, 
    epochs=30,
    workers=4,
    seed=SEED
)
print(f"✅ Word2Vec 완료 (Vocab: {len(w2v_model.wv.index_to_key):,}개)")

# 3-2. Doc2Vec 학습
print("\n🚀 Doc2Vec 학습 시작... (Tags: Method + Category)")
documents = []
for rid, tokens, method, cat in zip(df['레시피일련번호'], df['재료토큰'], df['요리방법별명'], df['요리종류별명_세분화']):
    # 조리법과 카테고리를 태그로 넣어 문맥에 반영
    words = list(tokens) + [f"METHOD_{method}"] * 2 + [f"CATEGORY_{cat}"] * 2
    tag = f"recipe_{rid}"
    documents.append(TaggedDocument(words=words, tags=[tag]))

d2v_model = Doc2Vec(
    documents,
    vector_size=100,
    window=30,
    min_count=3,
    # sample=1e-4,
    workers=4,
    dm=1, # DM 방식 권장
    epochs=50,
    seed=SEED
)
print(f"✅ Doc2Vec 완료 (Docs: {len(d2v_model.dv):,}개)")

🚀 Word2Vec 학습 시작... (Window=30, Min=3, Sample=1e-4)
✅ Word2Vec 완료 (Vocab: 5,425개)

🚀 Doc2Vec 학습 시작... (Tags: Method + Category)
✅ Doc2Vec 완료 (Docs: 130,735개)


In [9]:
from collections import Counter
import itertools

# 1. 전체 토큰(단어)을 하나로 모아서 개수 세기
all_tokens = list(itertools.chain.from_iterable(df['재료토큰']))
token_counts = Counter(all_tokens)

print(f"🍎 전체 고유 단어(토큰) 개수: {len(token_counts):,}개")
    
# 2. 빈도수 3회 이상인 단어 개수 (Word2Vec이 학습한 대상)
valid_tokens = {word: count for word, count in token_counts.items() if count >= 3}
print(f"✅ 3회 이상 등장한 단어 개수: {len(valid_tokens):,}개")

# 3. Word2Vec 모델의 벡터 개수
print(f"📊 실제 모델에 학습된 벡터 개수: {len(w2v_model.wv.index_to_key):,}개")

# 4. Dc22Vec 모델의 벡터 개수
print(f"📊 실제 모델에 학습된 벡터 개수: {len(d2v_model.dv.index_to_key):,}개")

🍎 전체 고유 단어(토큰) 개수: 19,089개
✅ 3회 이상 등장한 단어 개수: 5,425개
📊 실제 모델에 학습된 벡터 개수: 5,425개
📊 실제 모델에 학습된 벡터 개수: 130,735개


In [10]:
# === 학습 데이터 검증: 버린 단어 vs 챙긴 단어 ===

# 1. 전체 단어 카운트 다시 계산 (위에서 이미 하셨다면 생략 가능)
from collections import Counter
import itertools
all_tokens = list(itertools.chain.from_iterable(df['재료토큰']))
token_counts = Counter(all_tokens)

# 2. 분류 (생존 vs 탈락)
kept_tokens = [word for word, cnt in token_counts.items() if cnt >= 3]
dropped_tokens = [word for word, cnt in token_counts.items() if cnt < 3]

print(f"✅ 살아남은 재료 (예시 30개):")
print(kept_tokens[:30])
print("-" * 50)

print(f"🗑️ 버려진 재료 (예시 30개) - 오타나 너무 특이한 재료인지 보세요:")
print(dropped_tokens[:30])
print("-" * 50)

print(f"📊 요약: 전체 {len(token_counts):,}개 중 {len(kept_tokens):,}개 학습 (삭제율: {len(dropped_tokens)/len(token_counts)*100:.1f}%)")

✅ 살아남은 재료 (예시 30개):
['녹말가루', '브로콜리', '새우', '계란', '고추', '두부', '당근', '노른자', '참기름', '김가루', '후리가케', '날치', '참깨', '단무지', '무', '밥', '신김치', '물', '소금', '호두', '현미', '찹쌀', '간장', '마늘', '파', '후춧가루', '설탕', '찹쌀가루', '북어포', '깨소금']
--------------------------------------------------
🗑️ 버려진 재료 (예시 30개) - 오타나 너무 특이한 재료인지 보세요:
['그외의', '양겨자가루', '헤비위핑크림', '앤젤헤어', '우메보시', '호두아몬드', '중조', '맛있는찌개가필요할때', '박력쌀가루', '청상추', '단호박퓨레', '적양배추', '카스테라가루', '대나무가루', '설탕물에', '졸인', '칠리참치통조림', '짜장면발', '껍질까서', '포도씨유기름', '동글어묵', '초밥유부', '쵸콜릿', '잘익은묵은김치', '매콤맛굴소스', '맛있는밥엔', '곤약분말', '깨소슴', '돈까스용등심', '돼지살코기']
--------------------------------------------------
📊 요약: 전체 19,089개 중 5,425개 학습 (삭제율: 71.6%)


# 2) 인덱스 구축

In [11]:
# === [Cell 21] 인덱스 및 통계 구축 (수정됨) ===
print("📊 인덱싱 및 Baseline 통계 집계 중...")

name_to_ids = defaultdict(list)
method_map = {}
cat_map = {}

recipes_by_ingredient = defaultdict(list)
ing_cat_counts = defaultdict(Counter)
ing_method_counts = defaultdict(Counter)

# [NEW] 전체 데이터셋의 조리법/카테고리 분포 (Baseline 계산용)
total_method_counts = Counter()
total_cat_counts = Counter()

for _, row in df.iterrows():
    rid = row['레시피일련번호']
    name = row['요리명']
    method = row['요리방법별명']
    cat = row['요리종류별명_세분화']
    ings = row['재료토큰']
    
    name_to_ids[name].append(rid)
    method_map[rid] = method
    cat_map[rid] = cat
    
    # [NEW] 전체 카운트 집계
    total_method_counts[method] += 1
    total_cat_counts[cat] += 1
    
    for ing in ings:
        recipes_by_ingredient[ing].append(rid)
        ing_cat_counts[ing][cat] += 1
        ing_method_counts[ing][method] += 1

# [NEW] 전체 레시피 수 (확률 계산 분모)
TOTAL_RECIPES = len(df)

print("✅ 인덱싱 및 Baseline 통계 완료")
print(f"   - 전체 조리법 분포: {dict(total_method_counts.most_common(5))}")

📊 인덱싱 및 Baseline 통계 집계 중...
✅ 인덱싱 및 Baseline 통계 완료
   - 전체 조리법 분포: {'끓이기': 33596, '볶음': 32643, '무침': 15534, '부침': 9310, '조림': 8973}


# 3) 대체 로직

## 3-1) 요리종류별명_세분화 & 코사인 유사도 계산 

In [12]:
# === [Cell 23] Helper Functions (Lift 적용) ===

# [NEW] Lift(향상도) 기반 통계 점수 계산
# Lift > 1.0 : 해당 조리법/카테고리와 양의 상관관계 (추천 대상)
# Lift = 1.0 : 관계 없음 (소금, 물 등)
# Lift < 1.0 : 음의 상관관계 (안 어울림)
def get_stat_score(ingredient, target_key, ing_count_dict, total_count_dict, total_n, min_count=5):
    """
    ingredient: 재료명 (예: '월계수잎')
    target_key: 현재 요리의 조리법/카테고리 (예: '찜')
    ing_count_dict: 재료별 분포 (ing_method_counts)
    total_count_dict: 전체 분포 (total_method_counts)
    total_n: 전체 레시피 수 (TOTAL_RECIPES)
    """
    # 1. 재료가 해당 그룹에 등장한 횟수
    cnts = ing_count_dict.get(ingredient)
    if not cnts: return 0.0
    
    ing_target_count = cnts[target_key] # 월계수잎이 찜에 쓰인 횟수
    ing_total_count = sum(cnts.values()) # 월계수잎 전체 사용 횟수
    
    # 최소 등장 횟수 필터링 (노이즈 제거)
    if ing_total_count < min_count: return 0.0
    
    # 2. 조건부 확률 P(Method | Ingredient)
    # "월계수잎을 썼을 때 찜일 확률"
    prob_ing_context = ing_target_count / ing_total_count
    
    # 3. 전체 확률 P(Method)
    # "아무 요리나 뽑았을 때 찜일 확률" (Baseline)
    baseline_prob = total_count_dict[target_key] / total_n
    
    # 4. Lift 계산 (분모 0 방지)
    if baseline_prob == 0: return 0.0
    
    lift = prob_ing_context / baseline_prob
    return lift

# [KEEP] 코사인 유사도 계산 (그대로 유지)
def cos_sim(vec_a, vec_b):
    norm = (np.linalg.norm(vec_a) * np.linalg.norm(vec_b) + 1e-9)
    return max(0.0, float(np.dot(vec_a, vec_b) / norm))

## 3-2) 단일 대체

In [13]:
# === [Cell 25] 단일 대체 로직 (부분 일치 불용어 필터 적용) ===

def substitute_single(recipe_id, target_ing, stopwords=[], w_w2v=1.0, w_d2v=1.0, w_method=1.0, w_cat=1.0, topn=10):
    """
    Min-Max Norm & Lift 적용. 
    stopwords에 포함된 단어가 후보 재료명에 '포함'되어 있으면 제외합니다. (Contains)
    """
    row = df[df['레시피일련번호'] == recipe_id]
    if row.empty: return pd.DataFrame()
    row = row.iloc[0]
    
    current_method = row['요리방법별명']
    current_cat = row['요리종류별명_세분화']
    context_ings = row['재료토큰']
    tag = f"recipe_{recipe_id}"
    
    if target_ing not in w2v_model.wv:
        return pd.DataFrame()

    total_weight = w_w2v + w_d2v + w_method + w_cat
    if total_weight == 0: total_weight = 1.0

    vec_recipe = None
    if w_d2v > 0 and tag in d2v_model.dv:
        vec_recipe = d2v_model.dv[tag]
    
    candidates_raw = w2v_model.wv.most_similar(target_ing, topn=50)
    
    temp_results = []
    
    for cand, score_w2v in candidates_raw:
        # [필터링 1] 이미 있는 재료 제외
        if cand in context_ings: continue 
        
        # [필터링 2] 사용자 지정 불용어 부분 일치(Contains) 검사 (NEW)
        # 예: stopwords=["시판용"] -> cand="시판용된장" -> "시판용" in "시판용된장" -> True -> 제외
        if stopwords and any(stop in cand for stop in stopwords):
            continue
        
        # 1. W2V
        s_w2v = max(0.0, score_w2v)
        if s_w2v < 0.35: continue 
        
        # 2. D2V
        s_d2v = 0.0
        if w_d2v > 0 and vec_recipe is not None:
            rid_list = recipes_by_ingredient.get(cand, [])
            same_method_ids = [r for r in rid_list if method_map.get(r) == current_method]
            
            if len(same_method_ids) > 20:
                np.random.seed(42) 
                same_method_ids = np.random.choice(same_method_ids, 20, replace=False)
            
            if len(same_method_ids) > 0:
                sims = []
                for r in same_method_ids:
                    rt = f"recipe_{r}"
                    if rt in d2v_model.dv:
                        sims.append(cos_sim(vec_recipe, d2v_model.dv[rt]))
                if sims: s_d2v = np.mean(sims)

        # 3. Stats
        s_method = 0.0
        s_cat = 0.0
        if w_method > 0:
            s_method = get_stat_score(cand, current_method, ing_method_counts, total_method_counts, TOTAL_RECIPES)
        if w_cat > 0:
            s_cat = get_stat_score(cand, current_cat, ing_cat_counts, total_cat_counts, TOTAL_RECIPES)
            
        temp_results.append({
            "대체재료": cand,
            "raw_W2V": s_w2v,
            "raw_D2V": s_d2v,
            "raw_Method": s_method,
            "raw_Category": s_cat
        })
    
    if not temp_results: return pd.DataFrame()
    
    df_res = pd.DataFrame(temp_results)
    
    # (4) Min-Max Norm
    cols = ["raw_W2V", "raw_D2V", "raw_Method", "raw_Category"]
    norm_cols = ["W2V", "D2V", "Method", "Category"]
    
    for raw_col, norm_col in zip(cols, norm_cols):
        min_val = df_res[raw_col].min()
        max_val = df_res[raw_col].max()
        if max_val - min_val == 0:
            df_res[norm_col] = 0.5
        else:
            df_res[norm_col] = (df_res[raw_col] - min_val) / (max_val - min_val)

    # (5) 가중 합산
    df_res["최종점수"] = (
        (df_res["W2V"] * w_w2v) +
        (df_res["D2V"] * w_d2v) +
        (df_res["Method"] * w_method) +
        (df_res["Category"] * w_cat)
    ) / total_weight
    
    return df_res.sort_values("최종점수", ascending=False).head(topn).reset_index(drop=True)

print("✅ 단일 대체 로직 업데이트 (부분 일치 필터 적용)")

✅ 단일 대체 로직 업데이트 (부분 일치 필터 적용)


## 3-3) 다중 대체

In [14]:
# === [Cell 26] 다중 대체 로직 (부분 일치 불용어 필터 적용) ===

def substitute_multi(recipe_id, targets, stopwords=[], w_w2v=1.0, w_d2v=1.0, w_method=1.0, w_cat=1.0, beam_width=3, result_topn=3):
    """
    stopwords 리스트를 부분 일치(Contains)로 검사하여 다중 대체를 수행합니다.
    """
    row = df[df['레시피일련번호'] == recipe_id]
    if row.empty: return []
    row = row.iloc[0]
    
    current_method = row['요리방법별명']
    current_cat = row['요리종류별명_세분화']
    initial_context = row['재료토큰']
    
    tag = f"recipe_{recipe_id}"
    vec_recipe = None
    if w_d2v > 0 and tag in d2v_model.dv:
        vec_recipe = d2v_model.dv[tag]
    
    total_weight = w_w2v + w_d2v + w_method + w_cat
    if total_weight == 0: total_weight = 1.0

    beam = [(0.0, [], initial_context)]
    
    for target_ing in targets:
        next_beam = []
        if target_ing not in w2v_model.wv:
            for score, subs, ctx in beam:
                next_beam.append((score, subs + [target_ing], ctx))
            beam = next_beam
            continue

        for path_score, path_subs, path_ctx in beam:
            current_ctx_ing = [x for x in path_ctx if x != target_ing]
            candidates = w2v_model.wv.most_similar(target_ing, topn=20)
            
            temp_candidates = []
            
            for cand, sim_orig in candidates:
                if cand in current_ctx_ing or cand in path_subs: continue
                
                # [필터링] 부분 일치(Contains) 검사 (NEW)
                if stopwords and any(stop in cand for stop in stopwords):
                    continue 
                
                sim_orig = max(0.0, sim_orig)
                if sim_orig < 0.3: continue
                
                harmony_scores = [w2v_model.wv.similarity(cand, c) for c in current_ctx_ing if c in w2v_model.wv]
                sim_harmony = np.mean(harmony_scores) if harmony_scores else 0.0
                s_w2v = 0.5 * sim_orig + 0.5 * max(0.0, sim_harmony)
                
                s_d2v = 0.0
                if vec_recipe is not None:
                    rid_list = recipes_by_ingredient.get(cand, [])
                    same_method_ids = [r for r in rid_list if method_map.get(r) == current_method]
                    if len(same_method_ids) > 20:
                        np.random.seed(42)
                        same_method_ids = np.random.choice(same_method_ids, 20, replace=False)
                    
                    if same_method_ids is not None and len(same_method_ids) > 0:
                        sims = []
                        for r in same_method_ids:
                            rt = f"recipe_{r}"
                            if rt in d2v_model.dv:
                                sims.append(cos_sim(vec_recipe, d2v_model.dv[rt]))
                        if sims: s_d2v = np.mean(sims)
                
                s_method = 0.0
                s_cat = 0.0
                if w_method > 0:
                    s_method = get_stat_score(cand, current_method, ing_method_counts, total_method_counts, TOTAL_RECIPES)
                if w_cat > 0:
                    s_cat = get_stat_score(cand, current_cat, ing_cat_counts, total_cat_counts, TOTAL_RECIPES)

                temp_candidates.append({
                    "cand": cand,
                    "raw_w2v": s_w2v,
                    "raw_d2v": s_d2v,
                    "raw_method": s_method,
                    "raw_cat": s_cat
                })
            
            if not temp_candidates: continue

            df_temp = pd.DataFrame(temp_candidates)
            cols = ["raw_w2v", "raw_d2v", "raw_method", "raw_cat"]
            for col in cols:
                min_val = df_temp[col].min()
                max_val = df_temp[col].max()
                if max_val - min_val == 0:
                    df_temp[col + "_norm"] = 0.5
                else:
                    df_temp[col + "_norm"] = (df_temp[col] - min_val) / (max_val - min_val)
            
            for _, r in df_temp.iterrows():
                weighted_sum = (
                    (r["raw_w2v_norm"] * w_w2v) +
                    (r["raw_d2v_norm"] * w_d2v) +
                    (r["raw_method_norm"] * w_method) +
                    (r["raw_cat_norm"] * w_cat)
                ) / total_weight
                
                new_total_score = path_score + weighted_sum
                new_subs = path_subs + [r["cand"]]
                new_ctx = current_ctx_ing + [r["cand"]]
                next_beam.append((new_total_score, new_subs, new_ctx))
        
        next_beam.sort(key=lambda x: x[0], reverse=True)
        beam = next_beam[:beam_width]
    
    results = []
    for score, subs, _ in beam:
        avg_score = score / len(targets) if targets else 0.0
        results.append((subs, avg_score))
        
    return results[:result_topn]

print("✅ 다중 대체 로직 업데이트 (부분 일치 필터 적용)")

✅ 다중 대체 로직 업데이트 (부분 일치 필터 적용)


# 4) UI

In [51]:
# === [Cell 12] [Final UI] 불용어 필터 & 반응형 테이블 레이아웃 ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
import pandas as pd

# -------------------------------------------------------
# UI 구성
# -------------------------------------------------------
lbl_title = widgets.HTML("<h2>🍳 AI 식재료 대체 추천 대시보드</h2>")

input_dish = widgets.Text(description="요리명:", placeholder="예: 김치찌개", layout=widgets.Layout(width='40%'))
btn_search = widgets.Button(description="검색", button_style='info', icon='search', layout=widgets.Layout(width='15%'))
dropdown_recipe = widgets.Dropdown(description="레시피:", options=[], disabled=True, layout=widgets.Layout(width='99%'))
input_target = widgets.Text(description="타겟 재료:", placeholder="예: 돼지고기, 양파", disabled=True, layout=widgets.Layout(width='45%'))

# [NEW] 불용어/기피재료 입력칸
input_stopwords = widgets.Text(description="제외 단어:", placeholder="예: 약간, or (불용어) / 새우, 오이 (알레르기)", disabled=True, layout=widgets.Layout(width='45%'))

style = {'description_width': 'initial'}
s_w2v = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='맛·성질(W2V)', style=style)
s_d2v = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='문맥(D2V)', style=style)
s_meth = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='조리법', style=style)
s_cat = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.5, description='카테고리', style=style)

btn_run = widgets.Button(description="분석 실행", button_style='primary', disabled=True, icon='rocket', layout=widgets.Layout(width='100%'))
output_main = widgets.Output(layout=widgets.Layout(width='100%'))

# -------------------------------------------------------
# UI 업데이트 로직
# -------------------------------------------------------
def update_dashboard(recipe_id, target_input_str, stop_input_str, w_w2v, w_d2v, w_method, w_cat):
    
    # 타겟 파싱
    raw_targets = target_input_str.replace(',', ' ').split()
    targets = [t.strip() for t in raw_targets if t.strip()]
    
    # 불용어 파싱
    raw_stops = stop_input_str.replace(',', ' ').split()
    stopwords = [s.strip() for s in raw_stops if s.strip()]
    
    if not targets: return
    
    # 수식 및 필터 정보 표시
    total_w = w_w2v + w_d2v + w_method + w_cat
    if total_w == 0: total_w = 1.0
    
    stop_msg = f"🚫 <b>제외 필터:</b> {', '.join(stopwords)}" if stopwords else "🚫 <b>제외 필터:</b> 없음"
    
    info_md = f"""
    <div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px; margin-bottom: 15px; border-left: 5px solid #007bff;">
        <p style="font-family: sans-serif; font-size: 0.95em; margin: 0 0 5px 0; color: #555;">
            ℹ️ <b>분석 기준:</b> 맛({w_w2v}) + 문맥({w_d2v}) + 조리법({w_method}) + 카테고리({w_cat})
        </p>
        <p style="font-family: sans-serif; font-size: 0.95em; margin: 0; color: #d9534f;">
            {stop_msg}
        </p>
    </div>
    """
    display(HTML(info_md))
    
    # 1. 단일 대체 결과 출력
    cached_results = {}
    valid_targets = []
    
    # [Style Fix] 부모 컨테이너: flex-wrap으로 줄바꿈 허용, gap으로 간격 조정
    html_out = "<div style='display: flex; flex-wrap: wrap; gap: 20px; margin-bottom: 20px; width: 100%;'>"
    
    for t in targets:
        # [외부 로직 호출]
        res = substitute_single(recipe_id, t, stopwords, w_w2v, w_d2v, w_method, w_cat, topn=5)
        
        if not res.empty:
            cached_results[t] = res
            valid_targets.append(t)
            
            display_df = res[['대체재료', '최종점수']].copy()
            display_df.columns = ['추천 재료', '적합도']
            
            # [Style Fix] 테이블 스타일: width: 100% 추가 (부모 div에 꽉 차게)
            styler = display_df.style.format("{:.1%}", subset=["적합도"]) \
                        .background_gradient(cmap='Greens', subset=["적합도"]) \
                        .set_caption(f"<b>🔹 {t} 대체 추천</b>") \
                        .set_table_attributes('style="width:100%; border:1px solid #ccc; border-collapse:collapse; text-align:center;"') \
                        .hide(axis='index')
            
            # [Style Fix] 자식 div: flex: 1 (공간 균등 분할), min-width (너무 좁아지면 줄바꿈)
            # min-width를 30% 정도로 주면 3개까지는 한 줄, 4개부터는 다음 줄로 넘어감
            html_out += f"<div style='flex: 1; min-width: 30%;'>{styler.to_html()}</div>"
        else:
            # 실패 메시지도 동일한 레이아웃 적용
            html_out += f"<div style='flex: 1; min-width: 30%; padding:10px; border:1px solid #ddd; background:#fff0f0;'><b>{t}:</b> ❌ 대체 재료 없음 (필터링 됨)</div>"
            
    html_out += "</div>"
    display(HTML(html_out))
    
    # 2. 다중 대체 결과 출력 (전체 너비 사용)
    if len(targets) > 1 and valid_targets:
        display(Markdown("### 🧩 최적의 재료 조합 (다중 대체)"))
        multi_res = substitute_multi(recipe_id, targets, stopwords, w_w2v, w_d2v, w_method, w_cat, beam_width=3)
        
        if multi_res:
            m_df = pd.DataFrame([
                (f"{', '.join(subs)}", score) for subs, score in multi_res
            ], columns=['추천 조합', '종합 점수'])
            
            # 다중 대체 테이블도 100% 너비 적용
            display(
                m_df.style.format("{:.1%}", subset=['종합 점수']) \
                    .background_gradient(cmap='Blues', subset=['종합 점수']) \
                    .set_table_attributes('style="width:100%; text-align:center;"') \
                    .hide(axis='index')
            )

# -------------------------------------------------------
# 이벤트 핸들러
# -------------------------------------------------------
def on_search_click(b):
    output_main.clear_output(); dropdown_recipe.options = []
    dish = input_dish.value.strip()
    if not dish: return
    
    cands = df[df['요리명'] == dish]
    if cands.empty:
        cands = df[df['요리명'].str.contains(dish, na=False)]
    
    cands = cands.head(10).reset_index(drop=True)
    
    if cands.empty:
        with output_main: print("❌ 검색 결과 없음. 다른 검색어로 시도해보세요."); return
        
    opts = []
    for _, r in cands.iterrows():
        ing_sum = ', '.join(r['재료토큰'])
        preview_text = ing_sum[:100] + "..." if len(ing_sum) > 100 else ing_sum
        label = f"[{r['요리종류별명_세분화']}/{r['요리방법별명']}] {r['요리명']} (ID:{r['레시피일련번호']}) - {preview_text}"
        opts.append((label, r['레시피일련번호']))
    
    dropdown_recipe.options = opts
    dropdown_recipe.disabled = False
    input_target.disabled = False
    input_stopwords.disabled = False
    btn_run.disabled = False
    
    with output_main: print(f"✅ '{dish}' 검색 완료 ({len(cands)}건)")

def update_result(b=None):
    if dropdown_recipe.disabled: return
    recipe_id = dropdown_recipe.value
    target_str = input_target.value.strip()
    stop_str = input_stopwords.value.strip()
    
    if not target_str: return
    
    with output_main:
        clear_output(wait=True)
        update_dashboard(
            recipe_id, target_str, stop_str,
            s_w2v.value, s_d2v.value, s_meth.value, s_cat.value
        )

# 연결
btn_search.on_click(on_search_click)
btn_run.on_click(update_result)
for s in [s_w2v, s_d2v, s_meth, s_cat]: 
    s.observe(lambda c: update_result(), names='value')

ui = widgets.VBox([
    lbl_title,
    widgets.HBox([input_dish, btn_search], layout=widgets.Layout(width='100%')),
    dropdown_recipe,
    widgets.HBox([input_target, input_stopwords], layout=widgets.Layout(width='100%')),
    widgets.HTML("<b>⚖️ 가중치 설정</b>"),
    widgets.HBox([s_w2v, s_d2v, s_meth, s_cat], layout=widgets.Layout(width='100%')),
    btn_run,
    widgets.HTML("<hr>"),
    output_main
], layout=widgets.Layout(width='99%', padding='10px'))

display(ui)

# 5) 배포

In [15]:
# === [Export] 배포를 위한 필수 데이터 저장 ===
import pickle

# 1. 학습된 모델 저장
w2v_model.save("w2v.model")
d2v_model.save("d2v.model")

# 2. 데이터프레임 저장
df.to_csv("final_recipe_data.csv", index=False)

# 3. 통계 딕셔너리 저장
stats_data = {
    "method_map": method_map,
    "recipes_by_ingredient": recipes_by_ingredient,
    "ing_method_counts": ing_method_counts,
    "ing_cat_counts": ing_cat_counts,
    "total_method_counts": total_method_counts,
    "total_cat_counts": total_cat_counts,
    "TOTAL_RECIPES": TOTAL_RECIPES
}

with open("stats.pkl", "wb") as f:
    pickle.dump(stats_data, f)

print("✅ 파일 생성 완료! 폴더를 확인하세요.")

✅ 파일 생성 완료! 폴더를 확인하세요.
